In [62]:
import numpy as np
import pandas as pd
import re
import sys
import urllib
import random
from bs4 import BeautifulSoup

# Pokemon Notebook : A Case Study Exploring many Avenues'

## Tinh Nguyen 4/13

Exuse the large blocks of code. I calculated these by web crawling using a BeautifulSoup Python 2 http://pokemondb.net/type Code for output on _____/_____.ipynb
```
type_advantage_mat
    - X_ij = ( Type i Attack, Type j Defend ) multiplier of type
```

In [63]:
# Import dataset
pokekmon = pd.read_csv('Pokemon.csv')

In [64]:
type_advantage_mat = np.array([[ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
         1. ,  0.5,  0. ,  1. ,  1. ,  0.5,  1. ],
       [ 1. ,  0.5,  0.5,  1. ,  2. ,  2. ,  1. ,  1. ,  1. ,  1. ,  1. ,
         2. ,  0.5,  1. ,  0.5,  1. ,  2. ,  1. ],
       [ 1. ,  2. ,  0.5,  1. ,  0.5,  1. ,  1. ,  1. ,  2. ,  1. ,  1. ,
         1. ,  2. ,  1. ,  0.5,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  2. ,  0.5,  0.5,  1. ,  1. ,  1. ,  0. ,  2. ,  1. ,
         1. ,  1. ,  1. ,  0.5,  1. ,  1. ,  1. ],
       [ 1. ,  0.5,  2. ,  1. ,  0.5,  1. ,  1. ,  0.5,  2. ,  0.5,  1. ,
         0.5,  2. ,  1. ,  0.5,  1. ,  0.5,  1. ],
       [ 1. ,  0.5,  0.5,  1. ,  2. ,  0.5,  1. ,  1. ,  2. ,  2. ,  1. ,
         1. ,  1. ,  1. ,  2. ,  1. ,  0.5,  1. ],
       [ 2. ,  1. ,  1. ,  1. ,  1. ,  2. ,  1. ,  0.5,  1. ,  0.5,  0.5,
         0.5,  2. ,  0. ,  1. ,  2. ,  2. ,  0.5],
       [ 1. ,  1. ,  1. ,  1. ,  2. ,  1. ,  1. ,  0.5,  0.5,  1. ,  1. ,
         1. ,  0.5,  0.5,  1. ,  1. ,  0. ,  2. ],
       [ 1. ,  2. ,  1. ,  2. ,  0.5,  1. ,  1. ,  2. ,  1. ,  0. ,  1. ,
         0.5,  2. ,  1. ,  1. ,  1. ,  2. ,  1. ],
       [ 1. ,  1. ,  1. ,  0.5,  2. ,  1. ,  2. ,  1. ,  1. ,  1. ,  1. ,
         2. ,  0.5,  1. ,  1. ,  1. ,  0.5,  1. ],
       [ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  2. ,  2. ,  1. ,  1. ,  0.5,
         1. ,  1. ,  1. ,  1. ,  0. ,  0.5,  1. ],
       [ 1. ,  0.5,  1. ,  1. ,  2. ,  1. ,  0.5,  0.5,  1. ,  0.5,  2. ,
         1. ,  1. ,  0.5,  1. ,  2. ,  0.5,  0.5],
       [ 1. ,  2. ,  1. ,  1. ,  1. ,  2. ,  0.5,  1. ,  0.5,  2. ,  1. ,
         2. ,  1. ,  1. ,  1. ,  1. ,  0.5,  1. ],
       [ 0. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  2. ,
         1. ,  1. ,  2. ,  1. ,  0.5,  1. ,  1. ],
       [ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
         1. ,  1. ,  1. ,  2. ,  1. ,  0.5,  0. ],
       [ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  0.5,  1. ,  1. ,  1. ,  2. ,
         1. ,  1. ,  2. ,  1. ,  0.5,  1. ,  0.5],
       [ 1. ,  0.5,  0.5,  0.5,  1. ,  2. ,  1. ,  1. ,  1. ,  1. ,  1. ,
         1. ,  2. ,  1. ,  1. ,  1. ,  0.5,  2. ],
       [ 1. ,  0.5,  1. ,  1. ,  1. ,  1. ,  2. ,  0.5,  1. ,  1. ,  1. ,
         1. ,  1. ,  1. ,  2. ,  2. ,  0.5,  1. ]])

In [65]:
key = {'Bug': 11,
 'Dark': 15,
 'Dragon': 14,
 'Electric': 3,
 'Fairy': 17,
 'Fighting': 6,
 'Fire': 1,
 'Flying': 9,
 'Ghost': 13,
 'Grass': 4,
 'Ground': 8,
 'Ice': 5,
 'Normal': 0,
 'Poison': 7,
 'Psychic': 10,
 'Rock': 12,
 'Steel': 16,
 'Water': 2}

In [66]:
pokekmon[['Name', 'Type 1', 'Type 2']].values

array([['Bulbasaur', 'Grass', 'Poison'],
       ['Ivysaur', 'Grass', 'Poison'],
       ['Venusaur', 'Grass', 'Poison'],
       ..., 
       ['HoopaHoopa Confined', 'Psychic', 'Ghost'],
       ['HoopaHoopa Unbound', 'Psychic', 'Dark'],
       ['Volcanion', 'Fire', 'Water']], dtype=object)

# Markov Chains! Best team? 

In this section of the notebook, I'll be building a Markov Chain to predict the best team or msot synergistic pokemon. We'll consider:
    - Type advantages
    - Base stats

## Attack and Defense Multipliers

Each Pokemon has a type (some two). In battle, type advantage alone can decide the outcome. For example, Fire is not very effective against Water. These type advantages will be the basis of our Markov Chain! 

An arc from Pokemon $A$ to Pokemon $B$ implies that $B$ is a good partner for $A$. This scenario can be described in a few good ways
- If $A$ is suceptible to type $T_{atk}$, $B_{atk}$ is super effective against $T$
- If $A$ is suceptible to type $T_{atk}$, $B_{def}$ is immune to $T_{atk}$
- If $A_{atk}$ not effective against $T_{atk}$, $B_{atk}$ super effective against $T$

For each Pokemon, we will have a type multiplier array.
```
    {Pokemon: [[atk_mult, def_mult]_1, [atk_mult, def_mult]_2, ...]
```


In [67]:
def create_pokemon_multipliers(pokemon_type, type_chart, type_indices):
    """
        @param pokemon_type - dataframe containing Name-Type1-Type2
        @param type_chart   - nd.array type advantage matrix
        @param type_indices - {Type_Name: column/row in matrix}
        
        @return pokemon_multipliers (dict) - {'Pokemon_name': [tav1, tav2, ...]}
    """
    pokemon_multipliers = {}
    for tup in pokemon_type.itertuples():
        pokemon_name = tup.Name
        types = []
        if type(tup._2) == str:
            types.append(tup._2)
        if type(tup._3) == str:
            types.append(tup._3)
        multipliers = [np.ones((18,)), np.ones((18,))]
        
        for t in types:
            # offense
            multipliers[0] = np.multiply(multipliers[0], type_chart[type_indices[t], :])
            # defense
            multipliers[1] = np.multiply(multipliers[1], type_chart[:, type_indices[t]])
        pokemon_multipliers[pokemon] = multipliers
    
    return pokemon_multipliers

In [68]:
create_pokemon_multipliers(pokemon[['Name', 'Type 1', 'Type 2']], type_advantage_mat, key)

NameError: name 'pokemon' is not defined

In [61]:
for x in pokekmon[['Name', 'Type 1', 'Type 2']].itertuples():
    print(x)

Pandas(Index=0, Name='Bulbasaur', _2='Grass', _3='Poison')
Pandas(Index=1, Name='Ivysaur', _2='Grass', _3='Poison')
Pandas(Index=2, Name='Venusaur', _2='Grass', _3='Poison')
Pandas(Index=3, Name='VenusaurMega Venusaur', _2='Grass', _3='Poison')
Pandas(Index=4, Name='Charmander', _2='Fire', _3=nan)
Pandas(Index=5, Name='Charmeleon', _2='Fire', _3=nan)
Pandas(Index=6, Name='Charizard', _2='Fire', _3='Flying')
Pandas(Index=7, Name='CharizardMega Charizard X', _2='Fire', _3='Dragon')
Pandas(Index=8, Name='CharizardMega Charizard Y', _2='Fire', _3='Flying')
Pandas(Index=9, Name='Squirtle', _2='Water', _3=nan)
Pandas(Index=10, Name='Wartortle', _2='Water', _3=nan)
Pandas(Index=11, Name='Blastoise', _2='Water', _3=nan)
Pandas(Index=12, Name='BlastoiseMega Blastoise', _2='Water', _3=nan)
Pandas(Index=13, Name='Caterpie', _2='Bug', _3=nan)
Pandas(Index=14, Name='Metapod', _2='Bug', _3=nan)
Pandas(Index=15, Name='Butterfree', _2='Bug', _3='Flying')
Pandas(Index=16, Name='Weedle', _2='Bug', _3='P